### Importing Data and Necessary libraries

In [57]:
import pandas as pd
import numpy as np
df = pd.read_csv('netflix_titles.csv')
df.head(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,25-Sep-21,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,24-Sep-21,2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...
2,s100,TV Show,On the Verge,NaN,"Julie Delpy, Elisabeth Shue, Sarah Jones, Alex...","France, United States",7-Sep-21,2021,TV-MA,1 Season,"TV Comedies, TV Dramas","Four women — a chef, a single mom, an heiress ..."


### Exploring Data - Data Frame Data Type

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


### Data Cleaning - Identifying Columns with Missing Data

In [58]:
# Identify which columns have missing data

# Calculate the percentage of missing values for all columns in data frame 
for i in df.columns:
    null_rate = df[i].isna().sum() / len(df) * 100 
    if null_rate > 0 :
        print("{} null rate: {}%".format(i,round(null_rate,2)))

director null rate: 29.91%
cast null rate: 9.37%
country null rate: 9.44%
date_added null rate: 0.11%
rating null rate: 0.05%
duration null rate: 0.03%


### Data Cleaning - Replacing Null Values

In [52]:
# Replace missing values in 'country', 'cast', and 'director' with 'No Data'

df['country'].replace(np.nan, 'No Data', inplace=True)
df['cast'].replace(np.nan, 'No Data', inplace=True)
df['director'].replace(np.nan, 'No Data', inplace=True)

### Data Cleaning - Dropping Null Values

In [53]:
# Count the number of rows including null values
original_row_count = len(df)

# Drop rows with any null values from the DataFrame
df.dropna(inplace=True)

# Count Rows with null values that have been dropped
rows_dropped = original_row_count - len(df)
print(f"{rows_dropped} rows with null values have been dropped.")

17 rows with null values have been dropped.


### Data Cleaning - Dropping Duplicates Values

In [49]:
# Before dropping duplicates
original_row_count = len(df)

# Drop duplicate rows from the DataFrame
df.drop_duplicates(inplace=True)

# After: Duplicate rows have been dropped
duplicates_dropped = original_row_count - len(df)
print(f"{duplicates_dropped} duplicate rows have been dropped.")


0 duplicate rows have been dropped.


### Data Cleaning - Double-Checking Null Values

In [133]:
# Check for missing values in each column of the DataFrame

df.isnull().sum()

show_id         0
type            0
title           0
director        0
cast            0
country         0
date_added      0
release_year    0
rating          0
duration        0
listed_in       0
description     0
dtype: int64

### Data Cleaning - Converting Date Data Types

In [24]:
# Converting Date Data Types and Adding Month-Year Columns

# Convert the 'date_added' column to datetime data type
df["date_added"] = pd.to_datetime(df['date_added'])


### Data Cleaning - Adding Month-Year Columns

In [25]:
# Extract the month and create a new column 'month_added'
df['month_added']=df['date_added'].dt.month

# Extract the month name and create a new column 'month_name_added'
df['month_name_added']=df['date_added'].dt.month_name()

# Extract the year and create a new column 'year_added'
df['year_added'] = df['date_added'].dt.year

df.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,month_added,month_name_added,year_added
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,No Data,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",9,September,2021


In [113]:
#df.set_index('show_id', inplace = True)

### Data Cleaning - Adding Primary Country Column

In [176]:
# The 'country' column currently contains multiple countries separated by commas.

# Split the 'country' column using the ',' delimiter and create a new DataFrame
split_df = df['country'].str.split(',', expand=True)

# Assign the first element to a new column 'first_country'
df['first_country'] = split_df[0]

# Drop the original 'country' column if needed
df.drop(columns=['country'], inplace=True)

df.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,month_added,month_name_added,year_added,target_ages,first_country
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",9.0,September,2021.0,Teens,United States


### Data Cleaning - Renaming Countries

In [177]:
#Renaming top countries that have long names
df['first_country'].replace('United States', 'USA', inplace=True)
df['first_country'].replace('United Kingdom', 'UK',inplace=True)
df['first_country'].replace('South Korea', 'S. Korea',inplace=True)

### Data Manipulation - Converting TV Show Ratings to Target Age Groups

In [175]:
# Dictionary mapping TV show ratings to target age groups
ratings_ages = {
    'TV-PG': 'Older Kids',
    'TV-MA': 'Adults',
    'TV-Y7-FV': 'Older Kids',
    'TV-Y7': 'Older Kids',
    'TV-14': 'Teens',
    'R': 'Adults',
    'TV-Y': 'Kids',
    'NR': 'Adults',
    'PG-13': 'Teens',
    'TV-G': 'Kids',
    'PG': 'Older Kids',
    'G': 'Kids',
    'UR': 'Adults',
    'NC-17': 'Adults'
}

# Creating the new 'target_ages' column using the mapping from 'ratings_ages'
df['target_ages'] = df['rating'].replace(ratings_ages)

df.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,month_added,month_name_added,year_added,target_ages
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",9.0,September,2021.0,Teens


### Data Manipulation - Extracting Duration of Movies in Minutes

In [15]:
# Function to extract duration in minutes from the 'duration'
def extract_duration(row):
    if pd.notna(row['duration']) and 'min' in row['duration']:
        duration = row['duration'].split(' min')[0]
        return int(duration)
    return None

# Apply the function to create the 'movie_duration' column
df['movie_duration'] = df.apply(extract_duration, axis=1)

df.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,month_added,month_name_added,year_added,movie_duration
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,No Data,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",9,September,2021,90.0


### Data Manipulation - Extracting Number of Seasons per TV Show

In [16]:
# Function to extract the number of seasons from the 'duration'
def extract_duration(row):
    if pd.notna(row['duration']) and 'Season' in row['duration']:
        duration = row['duration'].split(' Season')[0]
        return int(duration)
    return None

# Apply the function to create the 'tv_show_seasons' column
df['tv_show_seasons'] = df.apply(extract_duration, axis=1)

df.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,month_added,month_name_added,year_added,movie_duration,tv_show_seasons
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,No Data,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",9,September,2021,90.0,NaN


### Exporting cleaned data file ready for Analysis in Tableau

In [122]:
df.to_csv('netflix_titles_clean.csv', index=True)